# **Module 3: AI tools**
## DAT410

### Group 29 
### David Laessker, 980511-5012, laessker@chalmers.se

### Oskar Palmgren, 010529-4714, oskarpal@chalmers.se



We hereby declare that we have both actively participated in solving every exercise. All solutions are entirely our own work, without having taken part of other solutions.

___


## 1) Article summary and take-aways

___
## 2) Implementation

In [7]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

In [8]:
# Read in the data
beijing_data = pd.read_csv("Beijing_labeled.csv")
chengdu_data = pd.read_csv('Chengdu_labeled.csv')
guangzhou_data = pd.read_csv('Guangzhou_labeled.csv')
shanghai_data = pd.read_csv('Shanghai_labeled.csv')
shenyang_data = pd.read_csv('Shenyang_labeled.csv')

In [5]:
beijing_data.head()

,season,DEWP,HUMI,PRES,TEMP,Iws,precipitation,cbwd_NE,cbwd_NW,cbwd_SE,PM_HIGH
0,4,-8.0,79.0,1026.0,-5.0,23.69,0.0,0,0,1,1.0
1,4,-11.0,85.0,1021.0,-9.0,105.93,1.1,0,0,1,0.0
2,4,-21.0,43.0,1030.0,-11.0,117.55,0.0,0,1,0,0.0
3,4,-25.0,33.0,1034.0,-12.0,39.35,0.0,1,0,0,0.0
4,4,-24.0,30.0,1034.0,-10.0,59.00,0.0,1,0,0,0.0


In [50]:
#preprocessing, extract features and labels from the datasets
beijing_copy = beijing_data.copy()
shenyang_copy = shenyang_data.copy()

features_beijing = beijing_copy.drop('PM_HIGH', axis=1)
labels_beijing = beijing_copy['PM_HIGH'].astype(int)

features_shenyang = shenyang_copy.drop('PM_HIGH', axis=1)
labels_shenyang = shenyang_copy['PM_HIGH'].astype(int)

#print(labels_beijing)


#combine the beijing and shenyang features and labels to one training dataset
Xtrain = pd.concat([features_beijing, features_shenyang])
Ytrain = pd.concat([labels_beijing, labels_shenyang])

#print(Xtrain)

In [ ]:
#TODO: 

class Classifier:
    def __init__(self):
        pass

    def fit(self):
        pass

    def predict(self):
        pass

    def score(self):
        pass

___
Just testing...

In [62]:
#Train the kmeans model

test_clusters = 2

kmeans = KMeans(n_clusters=test_clusters, random_state=0)
cluster_labels = kmeans.fit_predict(Xtrain)

#print(cluster_labels)

#Assign labels to each centroids
labels = np.zeros(test_clusters)
for i in range(test_clusters):
    
    indices = np.where(cluster_labels == i)[0]
    #print(indices)

    
    majority_label = Ytrain.iloc[indices].mode()[0]
    #print(majority_label)
    
    labels[i] = majority_label



features_guangzhou = guangzhou_data.copy().drop('PM_HIGH', axis=1) #Xtest
labels_guangzhou = guangzhou_data.copy()['PM_HIGH'].astype(int) #Ytest

predicted_clusters = kmeans.predict(features_guangzhou) #Ypredicted

#print(predicted_clusters)

score = [predicted_clusters == labels_guangzhou] #True or False at each index if the labels are correct

print(f'Accuracy: {np.mean(score)}')





c:\Users\P_Osk\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Accuracy: 0.9349112426035503
